In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import logging
logging.basicConfig(level=logging.INFO)

import warnings
warnings.simplefilter("ignore", ResourceWarning)

# Vectrix Demo 👨🏻‍💻
This notebook demonstrates the functions for importing data from various sources. 
Loading it into a VectorStore, and then using it to answer questions with a Retrieval Augemented Reasoning  🦜🔗 LangGraph.

## Importing Data
### 1. From a URL 🔗

**Web Crawling and Data Extraction Example**

This cell demonstrates the process of crawling a website, chunking the extracted content, and performing Named Entity Recognition (NER) using the Vectrix library.

#### Steps:

1. **Web Crawling**: 
   - Uses `vectrix.Crawler` to extract pages from "https://vectrix.ai"
   - Limits the crawl to a maximum of 20 pages

2. **Content Chunking**:
   - Utilizes `vectrix.Webchunker` to break down the extracted content
   - Chunks are created with a size of 500 characters and an overlap of 50 characters

3. **Named Entity Recognition**:
   - Employs `vectrix.Extract` with the 'Replicate' model
   - Uses the 'meta/meta-llama-3-70b-instruct' model for entity extraction

This example showcases a typical workflow for web data extraction and processing using the Vectrix library.


In [2]:
from vectrix.importers import WebCrawler

url = "https://vectrix.ai"

crawler = WebCrawler(url)
generator = crawler.extract()
scraped_pages = []

# Iterate through progress updates
for item in generator:
    if isinstance(item, dict):
        print(item)  # This is a progress update
    else:
        scraped_pages.extend(item)  # This is the final list of Documents
        break

# Now final_result contains the list of Documents
print(f"Total documents: {len(scraped_pages)}")

INFO:root:Starting extraction for site: vectrix.ai
2024-08-08 11:33:01,163 - root - INFO - Starting extraction for site: vectrix.ai
Fetching pages: 100%|##########| 1/1 [00:01<00:00,  1.23s/it]
INFO:root:Visiting links: ['https://vectrix.ai/offerings', 'https://vectrix.ai/blog', 'https://vectrix.ai/blog-post/google-deepminds-searchless-chess-engine---part-1', 'https://vectrix.ai/blog-post/advanced-applications-and-future-trends-in-entity-analysis', 'https://vectrix.ai/blog-post/understanding-large-and-small-language-models-key-differences-and-applications', 'https://vectrix.ai/blog-post/are-llm-benchmarks-and-leaderboards-just-marketing-tools', 'https://vectrix.ai/career', 'https://vectrix.ai/platform', 'https://vectrix.ai/contact-us', 'https://vectrix.ai/about-us']
2024-08-08 11:33:02,425 - root - INFO - Visiting links: ['https://vectrix.ai/offerings', 'https://vectrix.ai/blog', 'https://vectrix.ai/blog-post/google-deepminds-searchless-chess-engine---part-1', 'https://vectrix.ai/blog-

{'status': 'Started', 'pages_scraped': 0, 'total_links': 10}


Fetching pages: 100%|##########| 10/10 [00:05<00:00,  1.80it/s]
INFO:root:Pages processed: 10
2024-08-08 11:33:08,036 - root - INFO - Pages processed: 10
INFO:root:Links to visit: 13
2024-08-08 11:33:08,064 - root - INFO - Links to visit: 13
INFO:root:Visiting links: ['https://vectrix.ai/offerings/chat-ui', 'https://vectrix.ai/offerings/advice', 'https://vectrix.ai/offerings/projects', 'https://vectrix.ai/blog-post/the-basics-of-entity-analysis-beyond-just-identifying-names', 'https://vectrix.ai/blog-post/image-extraction-with-langchain-and-gemini', 'https://vectrix.ai/blog-post/image-extraction-with-langchain-and-gemini', 'https://vectrix.ai/blog-post/image-extraction-with-langchain-and-gemini', 'https://vectrix.ai/blog-post/image-extraction-with-langchain-and-gemini', 'https://vectrix.ai/job-list/software-engineer-front-end', 'https://vectrix.ai/job-list/open-application---create-your-own-dream-job', 'https://vectrix.ai/job-list/junior-ai-researcher', 'https://vectrix.ai/job-list/int

{'status': 'In progress', 'pages_scraped': 10, 'links_remaining': 13}


Fetching pages: 100%|##########| 13/13 [00:07<00:00,  1.84it/s]
INFO:root:Pages processed: 23
2024-08-08 11:33:15,180 - root - INFO - Pages processed: 23
INFO:root:Links to visit: 0
2024-08-08 11:33:15,226 - root - INFO - Links to visit: 0


{'status': 'In progress', 'pages_scraped': 23, 'links_remaining': 0}


INFO:root:SCRAPING FINISHED: Pages extracted 24
2024-08-08 11:33:15,713 - root - INFO - SCRAPING FINISHED: Pages extracted 24


{'status': 'Finished', 'pages_scraped': 24}
Total documents: 24


In [3]:
for page in scraped_pages:
    print(page.metadata['url'])
    print(page.page_content[:200])

https://vectrix.ai/offerings
Our Offerings
Project that challenge force
Working with Alex was interesting. He haveing into went beyond what asking provided such an amazing.
Project that challenge force
Working with Alex was inter
https://vectrix.ai/blog
Vectrix AI Insights: Trends, Innovations, and Updates
Explore the latest in AI with Vectrix Insights. Our blog covers trends, detailed looks at SLMs and LLMs, and real-world applications. Stay updated
https://vectrix.ai/blog-post/google-deepminds-searchless-chess-engine---part-1
Google DeepMind’s Searchless Chess Engine - Part 1
Dimitri Allaert
Introduction:
Discover how Google DeepMind’s Searchless Chess Engine, utilizing deep neural networks without traditional search metho
https://vectrix.ai/blog-post/advanced-applications-and-future-trends-in-entity-analysis
Advanced Applications and Future Trends in Entity Analysis
Dimitri Allaert
Entity analysis is transforming industries by turning raw data into actionable insights. This technolo

In [ ]:
from vectrix import Weaviate

weaviate = Weaviate()
weaviate.set_colleciton("Vectrix")

response = weaviate.list_metadata()
print(response)

#### Adding Data to Vector Store using Weaviate

This code demonstrates how to add extracted web data to a Weaviate vector store using the Vectrix library.

**Steps**:

1. **Import and Instantiate Weaviate**:
   - Import necessary modules from Vectrix
   - Create a Weaviate instance

2. **Create a Collection**:
   - Set up a new collection named 'Vectrix'
   - Use a local Ollama model for embeddings
   - Specify the model name and URL

3. **Prepare Data for Vectorization**:
   - Create `VectorDocument` objects for each extracted result
   - Include metadata such as title, URL, content, type, and NER information

4. **Add Data to Vector Store**:
   - Use `weaviate.add_data()` to insert the prepared documents

5. **Perform a Test Query**:
   - Get a retriever from the Weaviate instance
   - Execute a sample query to verify functionality

**Note:**
- You can remove a collection using `weaviate.remove_collection("Vectrix")`
- This example uses a local Ollama model, but you can use any compatible embedding model

This code showcases the process of storing and querying vectorized web data using Weaviate and Vectrix, enabling efficient semantic search and retrieval.


In [ ]:
from vectrix.db import Weaviate

# Instantiation of the Vector store
weaviate = Weaviate()

In [ ]:
from vectrix.db import Weaviate

# Instantiation of the Vector store
weaviate = Weaviate()

weaviate.remove_collection("Vectrix")

# Create a new collection to store the documents (you can also append them to an existsing one)
weaviate.create_collection(name='Vectrix', 
                           embedding_model='Ollama', # For this example we use a local Ollama model, but you can use any other model 
                           model_name="mxbai-embed-large:335m",
                           model_url="http://host.docker.internal:11434") # Ollama Embeddings model URL


weaviate.add_data(chunked_webdata)


# Perform a quick search to see if it works
retriever = weaviate.get_retriever()
retriever.invoke('Who are the Vectrix founders ?')

In [ ]:
weaviate.set_colleciton('Vectrix')
retriever = weaviate.get_retriever()
retriever.invoke('Who are the Vectrix founders ?')

### From OneDrive ☁
We can also connect with a OneDrive folder and download the information from there. After this process we can add the information to our Vector store in the same way. This way we can extract various kinds of documents like PDF, World, PowerPoint ...

Downloads files from OneDrive, processes them with Unstructured, and stores in Weaviate.

Steps:
1. Connect to OneDrive using Azure credentials
2. List and download all files
3. Process files with Unstructured importer
4. Add processed documents to Weaviate vector store
5. Close OneDrive connection

Requirements:
- vectrix library (OneDrive connector, Unstructured importer)
- Weaviate client
- Azure credentials as environment variables (AZURE_CLIENT_ID, AZURE_CLIENT_SECRET)

Note: Close OneDrive connection before uploading to vector store.

In [ ]:
from vectrix.connectors.onedrive import OneDrive
from vectrix.importers.unstructured import Unstructured

drive = OneDrive(azure_client_id = os.environ.get('AZURE_CLIENT_ID'), azure_client_secret = os.environ.get('AZURE_CLIENT_SECRET'))

drive.list_files()

In [ ]:
# Download all files in the drive, returns a list of downloaded files
downloaded_files = drive.download_files()
print(downloaded_files)

# Process teh documents using unstructured (we can't load RAW PDF files into a Vector store)
importer = Unstructured()
documents = importer.process_files(downloaded_files)

# Add the documents to the Vector store
weaviate = Weaviate()
weaviate.set_colleciton('Vectrix')
weaviate.add_data(documents)

# Close the drive, do this BEFORE you want to upload the files to a vector store
drive.close()